In [1]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
%matplotlib inline

In [ ]:
##variable declarations
nx = 41
ny = 41
nt = 10
nit = 50 
c = 1
dx = 2/(nx-1)
dy = 2/(ny-1)
x = np.linspace(0,2,nx)
y = np.linspace(0,2,ny)
X,Y = np.meshgrid(x,y)


##physical variables
rho = 1
nu = .1
F = 1
dt = .01

#initial conditions
u = np.zeros((ny,nx)) ##create a XxY vector of 0's
un = np.zeros((ny,nx)) ##create a XxY vector of 0's

v = np.zeros((ny,nx)) ##create a XxY vector of 0's
vn = np.zeros((ny,nx)) ##create a XxY vector of 0's

p = np.ones((ny,nx)) ##create a XxY vector of 0's
pn = np.ones((ny,nx)) ##create a XxY vector of 0's

b = np.zeros((ny,nx))

In [32]:
def ij_to_xy(x,y):
    return [((j/nx-1/2)*2,1/2-i/ny)*2]

In [2]:
def buildUpB(A, rho, dt, dx, dy, u, v):
    b = np.zeros_like(u)
    for i in range(5):
        for j in range(5):
            if i**2+j**2 < 25:
                
    
    b[1:-1,1:-1]=rho*(1/dt*((u[1:-1,2:]-u[1:-1,0:-2])/(2*dx)+(v[2:,1:-1]-v[0:-2,1:-1])/(2*dy))-\
                      ((u[1:-1,2:]-u[1:-1,0:-2])/(2*dx))**2-\
                      2*((u[2:,1:-1]-u[0:-2,1:-1])/(2*dy)*(v[1:-1,2:]-v[1:-1,0:-2])/(2*dx))-\
                      ((v[2:,1:-1]-v[0:-2,1:-1])/(2*dy))**2)
    return b

In [3]:
def presPoissPeriodic(p, dx, dy):
    pn = np.empty_like(p)
    
    for q in range(nit):
        pn = p.copy()
        p[1:-1,1:-1] = ((pn[1:-1,2:]+pn[1:-1,0:-2])*dy**2+(pn[2:,1:-1]+pn[0:-2,1:-1])*dx**2)/\
            (2*(dx**2+dy**2)) -\
            dx**2*dy**2/(2*(dx**2+dy**2))*b[1:-1,1:-1]
    return p

In [30]:
udiff = 1
stepcount = 0
A = 1

ims = []
fig = plt.figure(figsize = (11,7), dpi=100)
X, Y = np.meshgrid(x,y)

while udiff > .001:
    un = u.copy()
    vn = v.copy()
        
    b = buildUpB(rho, dt, dx, dy, u, v)
    p = presPoissPeriodic(p, dx, dy)

    u[1:-1,1:-1] = un[1:-1,1:-1]-\
        un[1:-1,1:-1]*dt/dx*(un[1:-1,1:-1]-un[1:-1,0:-2])-\
        vn[1:-1,1:-1]*dt/dy*(un[1:-1,1:-1]-un[0:-2,1:-1])-\
        dt/(2*rho*dx)*(p[1:-1,2:]-p[1:-1,0:-2])+\
        nu*(dt/dx**2*(un[1:-1,2:]-2*un[1:-1,1:-1]+un[1:-1,0:-2])+\
        dt/dy**2*(un[2:,1:-1]-2*un[1:-1,1:-1]+un[0:-2,1:-1]))+F*dt

    v[1:-1,1:-1] = vn[1:-1,1:-1]-\
        un[1:-1,1:-1]*dt/dx*(vn[1:-1,1:-1]-vn[1:-1,0:-2])-\
        vn[1:-1,1:-1]*dt/dy*(vn[1:-1,1:-1]-vn[0:-2,1:-1])-\
        dt/(2*rho*dy)*(p[2:,1:-1]-p[0:-2,1:-1])+\
        nu*(dt/dx**2*(vn[1:-1,2:]-2*vn[1:-1,1:-1]+vn[1:-1,0:-2])+\
        (dt/dy**2*(vn[2:,1:-1]-2*vn[1:-1,1:-1]+vn[0:-2,1:-1])))
    
    
    
    udiff = (np.sum(u)-np.sum(un))/np.sum(u)
    stepcount += 1
    plt.contourf(X,Y,p,alpha=0.5)
    # plt.colorbar()
    im = plt.quiver(X[::3, ::3], Y[::3, ::3], u[::3, ::3], v[::3, ::3])

    ims.append([im])
    
    
ani = animation.ArtistAnimation(fig, ims, interval=stepcount, blit=True)

plt.show()


In [ ]:
fig = plt.figure(figsize = (11,7), dpi=100)
X, Y = np.meshgrid(x,y)
plt.contourf(X,Y,p,alpha=0.5)

plt.colorbar()
plt.quiver(X[::3, ::3], Y[::3, ::3], u[::3, ::3], v[::3, ::3])
